# IA conversacional – ¡también conocido como Chatbot!

## Importar Librerias

In [ ]:
from dotenv import load_dotenv
from google import genai
from google.genai import types
import gradio as gr

## Cargar variables de entorno

In [ ]:
load_dotenv()

In [ ]:
gemini = genai.Client()
chat_client = gemini.chats.create(model="gemini-3-flash-preview")

In [ ]:
def chat(message, history):
    
    # 1. Enviamos el mensaje del usuario al chat
    stream = chat_client.send_message_stream(message)
    
    # 2. Impresión del historial (Debug)
    print("\n--- Historial Actualizado ---")
    for message in chat_client.get_history():
        print(f'role - {message.role}',end=": ")
        print(message.parts[0].text)
        
    # 3. Generación de respuesta (Streaming)
    partial_text = ""
    for chunk in stream:
        if chunk.text:
            partial_text += chunk.text
            yield partial_text
            


## ¡Y entonces entra la magia de Gradio!

In [ ]:
gr.ChatInterface(fn=chat).launch()

In [ ]:
system_message = """Eres un asistente útil en una tienda de ropa.
Debes tratar de alentar gentilmente al cliente a que pruebe los artículos que están en oferta.
Los sombreros tienen un 60 % de descuento y la mayoría de los demás artículos tienen un 50 % de descuento.
Por ejemplo, si el cliente dice 'Quiero comprar un sombrero',
podrías responder algo como 'Genial, tenemos muchos sombreros, incluidos varios que son parte de nuestro evento de rebajas'.
Anima al cliente a comprar sombreros si no está seguro de qué comprar."""

In [ ]:
generate_config = types.GenerateContentConfig(
    temperature=0.7,
    max_output_tokens=1000,
    system_instruction=system_message
)

gemini = genai.Client()
chat_client = gemini.chats.create(model="gemini-3-flash-preview", config=generate_config )

gr.ChatInterface(fn=chat).launch(inline=False)

In [ ]:
system_message += "\nSi el cliente pide zapatos, debes responder que los zapatos no están en oferta hoy, \
¡pero recuérdale al cliente que mire los sombreros!"

In [ ]:
generate_config = types.GenerateContentConfig(
    temperature=0.7,
    max_output_tokens=1000,
    system_instruction=system_message
)

gemini = genai.Client()
chat_client = gemini.chats.create(model="gemini-3-flash-preview", config=generate_config )

gr.ChatInterface(fn=chat).launch(inline=False)

In [ ]:
def chat(message, history):
    
    # Creamos una variable temporal para lo que enviaremos a Gemini
    prompt_a_enviar = message
    
    # Comprobamos si la palabra clave está en el mensaje (usamos lower() para ignorar mayúsculas)
    palabras_clave = ["cinturon", "cinturón"]
    
    if any(p in message.lower() for p in palabras_clave):
        # Añadimos la instrucción "del sistema" al final del mensaje
        instruccion_extra = (
            "\n\n[INSTRUCCIÓN DEL SISTEMA: Para mayor contexto, la tienda no vende cinturones, "
            "pero asegúrate de señalar otros artículos en oferta.]"
        )
        prompt_a_enviar = message + instruccion_extra

    # Nota: Gradio mostrará 'message' (limpio), pero Gemini recibe 'prompt_a_enviar' (con la instrucción)
    stream = chat_client.send_message_stream(prompt_a_enviar)
    
    partial_text = ""
    for chunk in stream:
        if chunk.text:
            partial_text += chunk.text
            yield partial_text

In [ ]:
generate_config = types.GenerateContentConfig(
    temperature=0.7,
    max_output_tokens=1000,
    system_instruction=system_message
)

gemini = genai.Client()
chat_client = gemini.chats.create(model="gemini-3-flash-preview", config=generate_config )

gr.ChatInterface(fn=chat).launch(inline=False)

## Ollama

In [ ]:
import ollama

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = ollama.chat(model="llama3.2", messages=messages)
    return response['message']['content']

gr.ChatInterface(fn=chat).launch(inline=False)